# Benchmark d'Accélération GPU vs CPU
## Régression Logistique : quand le CPU bat le GPU

---

### Objectifs

Ce notebook démontre que **l'utilisation d'un GPU n'est pas toujours avantageuse** : pour des modèles simples comme la régression logistique, le CPU peut être plus rapide que le GPU.

**Points clés :**
- Mesure précise du temps d'exécution CPU vs GPU sur un modèle de régression logistique
- Utilisation de `torch.profiler` pour analyser les performances
- Calcul du **Speed Up** (rapport de performance)
- **Conclusion :** Overhead de transfert CPU↔GPU et faible parallélisme → CPU souvent gagnant

---

### Contexte : Modèles simples vs modèles complexes

Les GPU excellent sur les opérations massivement parallèles (réseaux profonds, gros batchs). Pour un modèle linéaire avec peu de paramètres et des données de taille modérée, le coût de transfert des données vers le GPU et le lancement des kernels CUDA peut dépasser le gain de calcul.

## 1 Installation des Dépendances

Installez les bibliothèques nécessaires si ce n'est pas déjà fait :

In [1]:
!pip install torch numpy matplotlib

## 2 Imports et Vérification GPU

In [2]:
import torch
import torch.nn as nn
import time
import statistics
import numpy as np
from torch.profiler import profile, record_function, ProfilerActivity

# Vérification de la disponibilité du GPU
print("="*70)
print(" VÉRIFICATION DE L'ENVIRONNEMENT")
print("="*70)
print(f"PyTorch version : {torch.__version__}")
print(f"CUDA disponible : {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU détecté : {torch.cuda.get_device_name(0)}")
    print(f"Mémoire GPU : {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    print(f"CUDA version : {torch.version.cuda}")
else:
    print("  Aucun GPU détecté - Le benchmark ne comparera que le CPU")
print("="*70)

 VÉRIFICATION DE L'ENVIRONNEMENT
PyTorch version : 2.9.0+cpu
CUDA disponible : False
  Aucun GPU détecté - Le benchmark ne comparera que le CPU


## 3 Configuration du Benchmark

### Paramètres importants :

- **NUM_SAMPLES** : Nombre d'échantillons (taille du dataset)
- **NUM_FEATURES** : Nombre de features (dimension d'entrée)
- **BATCH_SIZE** : Taille des batchs pour l'inférence
- **NUM_RUNS** : Nombre d'exécutions pour statistiques robustes

In [3]:
# Configuration
NUM_SAMPLES = 10000
NUM_FEATURES = 100
NUM_RUNS = 3
BATCH_SIZE_CPU = 256
BATCH_SIZE_GPU = 512

print(f" Configuration du benchmark :")
print(f"   • Nombre d'échantillons : {NUM_SAMPLES}")
print(f"   • Nombre de features : {NUM_FEATURES}")
print(f"   • Batch size CPU : {BATCH_SIZE_CPU}")
print(f"   • Batch size GPU : {BATCH_SIZE_GPU}")
print(f"   • Runs par device : {NUM_RUNS}")

 Configuration du benchmark :
   • Nombre d'échantillons : 10000
   • Nombre de features : 100
   • Batch size CPU : 256
   • Batch size GPU : 512
   • Runs par device : 3


## 4 Préparation des Données

Génération de données synthétiques pour un problème de classification binaire (régression logistique).

In [4]:
# Données synthétiques : classification binaire
np.random.seed(42)
X_np = np.random.randn(NUM_SAMPLES, NUM_FEATURES).astype(np.float32)
y_np = (X_np[:, 0] + 0.5 * X_np[:, 1] + np.random.randn(NUM_SAMPLES) * 0.3 > 0).astype(np.int64)

print(f" Dataset créé : {X_np.shape[0]} échantillons, {X_np.shape[1]} features")
print(f" Répartition des classes : {np.bincount(y_np)}")
print(f"\nExemple (5 premières lignes, 3 features) :")
print(X_np[:5, :3])

 Dataset créé : 10000 échantillons, 100 features
 Répartition des classes : [5007 4993]

Exemple (5 premières lignes, 3 features) :
[[ 0.49671414 -0.1382643   0.64768857]
 [-1.4153707  -0.42064533 -0.34271452]
 [ 0.35778737  0.5607845   1.0830512 ]
 [-0.828995   -0.560181    0.7472936 ]
 [-1.5944277  -0.599375    0.0052437 ]]


## 5 Définition du Modèle (Régression Logistique)

Régression logistique = une couche linéaire + sigmoid. Modèle très léger (peu de paramètres), peu adapté au parallélisme massif du GPU.

In [5]:
class LogisticRegressionPyTorch(nn.Module):
    """Régression logistique : Linear + Sigmoid."""
    def __init__(self, in_features, num_classes=1):
        super().__init__()
        self.linear = nn.Linear(in_features, num_classes)

    def forward(self, x):
        return torch.sigmoid(self.linear(x)).squeeze(-1)


model = LogisticRegressionPyTorch(NUM_FEATURES)
num_params = sum(p.numel() for p in model.parameters())
print(f" Modèle : LogisticRegression (PyTorch)")
print(f" Nombre de paramètres : {num_params}")
print(f" Modèle chargé avec succès.")

 Modèle : LogisticRegression (PyTorch)
 Nombre de paramètres : 101
 Modèle chargé avec succès.


## 6 Fonction de Benchmark avec Profilage

### Points clés :

1. **Warm-up** : Le premier passage sur GPU initialise les kernels CUDA (overhead)
2. **torch.profiler** : Capture les métriques CPU et CUDA
3. **record_function** : Isole l'opération critique
4. **Export de traces** : Pour visualisation dans Chrome

In [6]:
def run_benchmark(device_name, batch_size, X_tensor, run_number=1):
    """
    Exécute un benchmark d'inférence sur le device spécifié avec profilage.

    Args:
        device_name: 'cpu' ou 'cuda'
        batch_size: Taille du batch pour l'inférence
        X_tensor: Données déjà sur le bon device
        run_number: Numéro de l'exécution (pour affichage)

    Returns:
        float: Temps d'exécution en secondes
    """
    print(f"\n{'─' * 70}")
    print(f" RUN #{run_number} - Device : {device_name.upper()} (Batch size: {batch_size})")
    print(f"{'─' * 70}")

    device = torch.device(device_name)
    model.to(device)
    model.eval()

    # WARM-UP
    if run_number == 1:
        print("   Warm-up (chauffe du GPU/CPU)...")
        with torch.no_grad():
            _ = model(X_tensor[:batch_size])
        print("   Warm-up terminé")

    activities = [ProfilerActivity.CPU]
    if device_name == 'cuda':
        activities.append(ProfilerActivity.CUDA)

    print("   Démarrage du profilage...")
    start_time = time.time()

    with torch.no_grad():
        with profile(
            activities=activities,
            record_shapes=True,
            profile_memory=True,
            with_stack=False
        ) as prof:
            with record_function("model_inference"):
                for i in range(0, len(X_tensor), batch_size):
                    batch = X_tensor[i:i + batch_size]
                    _ = model(batch)

    end_time = time.time()
    total_time = end_time - start_time

    print(f"\n   Temps d'exécution : {total_time:.4f} secondes")

    print(f"\n   Analyse du profiler ({device_name.upper()}):")
    for evt in prof.key_averages():
        if evt.key == "model_inference":
            print(f"     • Temps CPU total : {evt.cpu_time_total / 1000:.2f} ms")
            if device_name == 'cuda' and hasattr(evt, 'self_cuda_time_total'):
                print(f"     • Temps CUDA total : {evt.self_cuda_time_total / 1000:.2f} ms")

    if run_number == 1:
        print(f"\n   Top 5 des opérations ({device_name.upper()}):")
        sort_key = "self_cuda_time_total" if device_name == 'cuda' else "cpu_time_total"
        print(prof.key_averages().table(sort_by=sort_key, row_limit=5))
        trace_file = f"trace_lr_{device_name}_run{run_number}.json"
        prof.export_chrome_trace(trace_file)
        print(f"\n   Trace exportée : {trace_file}")
        print(f"     → Visualiser dans Chrome : chrome://tracing")

    return total_time

## 7 Exécution du Benchmark CPU

### Pourquoi 3 runs ?
- Éliminer les variations dues au système d'exploitation
- Calculer des statistiques fiables (médiane, écart-type)
- Détecter les outliers

In [7]:
print("="*70)
print("  PHASE 1 : BENCHMARK CPU")
print("="*70)

X_cpu = torch.from_numpy(X_np)

cpu_times = []
for i in range(NUM_RUNS):
    cpu_time = run_benchmark('cpu', BATCH_SIZE_CPU, X_cpu, run_number=i+1)
    cpu_times.append(cpu_time)
    if i < NUM_RUNS - 1:
        time.sleep(1)

cpu_time_median = statistics.median(cpu_times)
cpu_time_mean = statistics.mean(cpu_times)
cpu_time_stdev = statistics.stdev(cpu_times) if len(cpu_times) > 1 else 0

print(f"\n Statistiques CPU ({NUM_RUNS} runs):")
print(f"   • Temps médian : {cpu_time_median:.4f} s")
print(f"   • Temps moyen : {cpu_time_mean:.4f} s")
print(f"   • Écart-type : {cpu_time_stdev:.4f} s")
print(f"   • Tous les temps : {[f'{t:.4f}s' for t in cpu_times]}")

  PHASE 1 : BENCHMARK CPU

──────────────────────────────────────────────────────────────────────
 RUN #1 - Device : CPU (Batch size: 256)
──────────────────────────────────────────────────────────────────────
   Warm-up (chauffe du GPU/CPU)...
   Warm-up terminé
   Démarrage du profilage...

   Temps d'exécution : 3.0640 secondes

   Analyse du profiler (CPU):
     • Temps CPU total : 2.17 ms

   Top 5 des opérations (CPU):
----------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                  Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
----------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
       model_inference        35.42%     770.077us       100.00%       2.174ms       2.174ms          64 B     -78.06 KB             1  
          aten::linear 

## 8 Exécution du Benchmark GPU

Sur GPU : transfert CPU→GPU des données + inférence. L'overhead peut rendre le GPU plus lent que le CPU pour un modèle aussi simple.

In [8]:
if torch.cuda.is_available():
    print("\n" + "="*70)
    print(" PHASE 2 : BENCHMARK GPU")
    print("="*70)
    print(f" GPU détecté : {torch.cuda.get_device_name(0)}")
    print(f" Mémoire GPU disponible : {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

    X_gpu = torch.from_numpy(X_np).cuda()

    gpu_times = []
    for i in range(NUM_RUNS):
        if i > 0:
            torch.cuda.empty_cache()
            time.sleep(1)
        gpu_time = run_benchmark('cuda', BATCH_SIZE_GPU, X_gpu, run_number=i+1)
        gpu_times.append(gpu_time)

    gpu_time_median = statistics.median(gpu_times)
    gpu_time_mean = statistics.mean(gpu_times)
    gpu_time_stdev = statistics.stdev(gpu_times) if len(gpu_times) > 1 else 0

    print(f"\n Statistiques GPU ({NUM_RUNS} runs):")
    print(f"   • Temps médian : {gpu_time_median:.4f} s")
    print(f"   • Temps moyen : {gpu_time_mean:.4f} s")
    print(f"   • Écart-type : {gpu_time_stdev:.4f} s")
    print(f"   • Tous les temps : {[f'{t:.4f}s' for t in gpu_times]}")
else:
    print("\n Aucun GPU détecté - Impossible de continuer le benchmark GPU")


 Aucun GPU détecté - Impossible de continuer le benchmark GPU


## 9 Calcul du Speed Up et Analyse

### Formule du Speed Up :

$$\text{Speed Up} = \frac{T_{\text{CPU}}}{T_{\text{GPU}}}$$

**Interprétation :**
- Speed Up > 1 : GPU plus rapide
- Speed Up = 1 : Performances égales
- **Speed Up < 1 : CPU plus rapide** → cas typique pour la régression logistique !

In [9]:
if torch.cuda.is_available():
    print("\n" + "="*70)
    print(" RÉSULTATS FINAUX")
    print("="*70)

    speed_up = cpu_time_median / gpu_time_median

    print(f"\n  Temps d'exécution (médiane):")
    print(f"   • CPU : {cpu_time_median:.4f} s")
    print(f"   • GPU : {gpu_time_median:.4f} s")
    print(f"\n SPEED UP : {speed_up:.2f}x")

    time_saved = abs(cpu_time_median - gpu_time_median)
    percent_faster = abs(cpu_time_median - gpu_time_median) / max(cpu_time_median, gpu_time_median) * 100

    print(f"\n  Comparaison:")
    print(f"   • Écart : {time_saved:.4f} s")
    if speed_up >= 1:
        print(f"   • GPU {percent_faster:.1f}% plus rapide")
    else:
        print(f"   • CPU {percent_faster:.1f}% plus rapide")

    throughput_cpu = NUM_SAMPLES / cpu_time_median
    throughput_gpu = NUM_SAMPLES / gpu_time_median
    print(f"\n  Throughput (échantillons/seconde):")
    print(f"   • CPU : {throughput_cpu:.0f} samples/s")
    print(f"   • GPU : {throughput_gpu:.0f} samples/s")

    print(f"\n{'─' * 70}")
    if speed_up < 1:
        print(" CONCLUSION : CPU plus rapide que le GPU !")
        print(f"   Pour la régression logistique, le CPU est {1/speed_up:.1f}x plus rapide.")
        print("   L'overhead de transfert CPU→GPU et le faible parallélisme du modèle")
        print("   ne justifient pas l'utilisation du GPU. GPUs ≠ toujours mieux.")
    elif speed_up < 1.5:
        print(" CONCLUSION : Gain GPU marginal.")
        print("   Pour ce modèle simple, le GPU n'apporte qu'un gain limité.")
    else:
        print(" CONCLUSION : GPU plus rapide.")
        print(f"   Le GPU est {speed_up:.1f}x plus rapide (cas moins fréquent pour la LR).")
    print("="*70)
else:
    print("\n  Impossible de calculer le Speed Up sans GPU")


  Impossible de calculer le Speed Up sans GPU


## 10 Visualisation des Résultats

In [10]:
import matplotlib.pyplot as plt

if torch.cuda.is_available():
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    devices = ['CPU', 'GPU']
    times = [cpu_time_median, gpu_time_median]
    colors = ['#FF6B6B', '#4ECDC4']

    bars = axes[0].bar(devices, times, color=colors, alpha=0.7, edgecolor='black')
    axes[0].set_ylabel('Temps (secondes)', fontsize=12)
    axes[0].set_title('Comparaison CPU vs GPU (Régression Logistique)', fontsize=14, fontweight='bold')
    axes[0].grid(axis='y', alpha=0.3)
    for bar, t in zip(bars, times):
        axes[0].text(bar.get_x() + bar.get_width()/2., bar.get_height(),
                    f'{t:.4f}s', ha='center', va='bottom', fontsize=11, fontweight='bold')

    throughputs = [throughput_cpu, throughput_gpu]
    bars2 = axes[1].bar(devices, throughputs, color=colors, alpha=0.7, edgecolor='black')
    axes[1].set_ylabel('Échantillons par seconde', fontsize=12)
    axes[1].set_title('Throughput (samples/s)', fontsize=14, fontweight='bold')
    axes[1].grid(axis='y', alpha=0.3)
    for bar, tp in zip(bars2, throughputs):
        axes[1].text(bar.get_x() + bar.get_width()/2., bar.get_height(),
                    f'{tp:.0f}', ha='center', va='bottom', fontsize=11, fontweight='bold')

    plt.tight_layout()
    plt.savefig('benchmark_logistic_regression.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("\n Graphiques sauvegardés : benchmark_logistic_regression.png")
else:
    print(" Aucun GPU - pas de visualisation comparative.")

 Aucun GPU - pas de visualisation comparative.


## Résumé et Notes Techniques

### Points clés à retenir :

1. **GPU ≠ toujours mieux** : Pour des modèles simples (régression logistique, petits réseaux), le CPU peut être plus rapide.
2. **Overhead GPU** : Transfert des données CPU→GPU et lancement des kernels CUDA ont un coût fixe.
3. **Quand utiliser le GPU** : Modèles profonds, gros batchs, beaucoup de calculs parallélisables (ex. BERT, ResNet).
4. **Quand rester sur CPU** : Modèles linéaires, petits datasets, faible complexité.

### Facteurs qui favorisent le CPU sur la régression logistique :

- Peu de paramètres → peu d'opérations à paralléliser
- Données déjà en RAM → pas de transfert
- Bibliothèques CPU (e.g. scikit-learn, numpy) très optimisées pour ce cas

### Pour aller plus loin :

- Tester avec des datasets plus grands (100k+ samples) pour voir si le GPU finit par gagner
- Comparer avec scikit-learn `LogisticRegression` (CPU uniquement, souvent très rapide)
- Visualiser les traces dans Chrome : `chrome://tracing`

## Références

- [PyTorch Profiler Documentation](https://pytorch.org/docs/stable/profiler.html)
- [When to use GPU vs CPU for ML](https://developer.nvidia.com/blog/when-to-use-gpu-acceleration/)
- [CUDA Best Practices](https://docs.nvidia.com/cuda/cuda-c-best-practices-guide/)

---

**Notebook :** P3 - Régression Logistique (GPU pas toujours mieux que CPU)  